In [1]:
%env CUDA_VISIBLE_DEVICES=1,2

import os
import sys
import numpy as np

# Add the parent directory to sys.path
sys.path.append(os.path.dirname(os.path.realpath(os.path.abspath(""))))

from unet.trainer import DeadwoodTrainer

env: CUDA_VISIBLE_DEVICES=1,2


In [3]:
experiment_name = "50k_512px_60epochs_3fold_bf1_tune_fn"
config = DeadwoodTrainer.default_config
trainer = DeadwoodTrainer(experiment_name, config=config)

In [4]:
trainer.run()

AttributeError: 'str' object has no attribute 'mkdir'

In [ ]:
register_df = pd.read_csv(register_file)
dataset = DeadwoodDataset(
    register_df=register_df,
    no_folds=no_folds,
    random_seed=random_seed,
    bins=train_bins,
    test_size=0,
)

In [ ]:
def initialize_experiment(run_name: str):
    # initialize a new experiment run
    experiment = wandb.init(
        project="standing-deadwood-unet-pro",
        resume="allow",
        name=run_name,
    )
    experiment.config.update(
        dict(
            data=dict(
                register_file=register_file,
                no_folds=no_folds,
                random_seed=random_seed,
                batch_size=batch_size,
                save_checkpoint=save_checkpoint,
                balaning_factor=balaning_factor,
                epochs=epochs,
                epoch_train_samples=epoch_train_samples,
                train_bins=train_bins,
            ),
            model=dict(
                epochs=epochs,
                learning_rate=learning_rate,
                amp=amp,
                weight_decay=weight_decay,
                momentum=momentum,
                gradient_clipping=gradient_clipping,
            ),
        )
    )
    return experiment